In [1]:
!pip install kagglehub
!pip install opencv-python
!pip install mediapipe
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("niharika41298/yoga-poses-dataset")

print("Path to dataset files:", dataset_path)

Path to dataset files: C:\Users\gheri\.cache\kagglehub\datasets\niharika41298\yoga-poses-dataset\versions\1


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def calculate_angle(a, b, c):
    """Calculate the angle between three points"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1, 1)))

def extract_features(image_path):
    """Extract both image and biomechanical features from a single image"""
    img = cv2.imread(image_path)
    if img is None:
        return None
    
    # Process image with MediaPipe
    results = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    if not results.pose_landmarks:
        return None
    
    # Get landmarks
    landmarks = results.pose_landmarks.landmark
    
    # Calculate key angles
    points = {
        'left_shoulder': [landmarks[11].x, landmarks[11].y],
        'right_shoulder': [landmarks[12].x, landmarks[12].y],
        'left_elbow': [landmarks[13].x, landmarks[13].y],
        'right_elbow': [landmarks[14].x, landmarks[14].y],
        'left_wrist': [landmarks[15].x, landmarks[15].y],
        'right_wrist': [landmarks[16].x, landmarks[16].y],
        'left_hip': [landmarks[23].x, landmarks[23].y],
        'right_hip': [landmarks[24].x, landmarks[24].y],
        'left_knee': [landmarks[25].x, landmarks[25].y],
        'right_knee': [landmarks[26].x, landmarks[26].y],
        'left_ankle': [landmarks[27].x, landmarks[27].y],
        'right_ankle': [landmarks[28].x, landmarks[28].y]
    }
    
    angles = {
        'left_elbow': calculate_angle(points['left_shoulder'], points['left_elbow'], points['left_wrist']),
        'right_elbow': calculate_angle(points['right_shoulder'], points['right_elbow'], points['right_wrist']),
        'left_shoulder': calculate_angle(points['left_elbow'], points['left_shoulder'], points['left_hip']),
        'right_shoulder': calculate_angle(points['right_elbow'], points['right_shoulder'], points['right_hip']),
        'left_hip': calculate_angle(points['left_shoulder'], points['left_hip'], points['left_knee']),
        'right_hip': calculate_angle(points['right_shoulder'], points['right_hip'], points['right_knee']),
        'left_knee': calculate_angle(points['left_hip'], points['left_knee'], points['left_ankle']),
        'right_knee': calculate_angle(points['right_hip'], points['right_knee'], points['right_ankle']),
        'spine': calculate_angle(points['left_shoulder'], points['left_hip'], points['left_ankle'])
    }
    
    # Prepare image for CNN
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    
    return {
        'image': img_normalized,
        'angles': list(angles.values()),
        'landmarks': points
    }

def process_dataset(dataset_path):
    """Process entire dataset and save features"""
    data = []
    pose_classes = [d for d in os.listdir(dataset_path) 
                   if os.path.isdir(os.path.join(dataset_path, d))]
    
    for pose_class in pose_classes:
        class_dir = os.path.join(dataset_path, pose_class)
        image_files = [f for f in os.listdir(class_dir) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for img_file in tqdm(image_files, desc=f"Processing {pose_class}"):
            img_path = os.path.join(class_dir, img_file)
            features = extract_features(img_path)
            
            if features is not None:
                features['class'] = pose_class
                data.append(features)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Save processed data
    df.to_pickle('yoga_dataset_processed.pkl')
    
    # Calculate pose standards
    pose_standards = df.groupby('class')['angles'].apply(
        lambda x: {
            'median': np.median(np.vstack(x), axis=0),
            'std': np.std(np.vstack(x), axis=0)
        }
    ).to_dict()
    
    with open('pose_standards.pkl', 'wb') as f:
        pickle.dump(pose_standards, f)
    
    return df, pose_standards

if __name__ == "__main__":
    dataset_path = r"C:\Users\gheri\.cache\kagglehub\datasets\niharika41298\yoga-poses-dataset\versions\1\DATASET\TRAIN"
    df, standards = process_dataset(dataset_path)

Processing warrior2: 100%|██████████| 252/252 [00:23<00:00, 10.53it/s]


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

def create_hybrid_model(num_classes=5):
    """Create a model that combines CNN and biomechanical features"""
    # Image branch (CNN)
    image_input = layers.Input(shape=(224, 224, 3), name='image_input')
    
    x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Angle branch
    angle_input = layers.Input(shape=(9,), name='angle_input')
    a = layers.Dense(32, activation='relu')(angle_input)
    
    # Combined features
    combined = layers.concatenate([x, a])
    
    # Classifier
    z = layers.Dense(128, activation='relu')(combined)
    z = layers.Dropout(0.5)(z)
    output = layers.Dense(num_classes, activation='softmax')(z)
    
    return models.Model(inputs=[image_input, angle_input], outputs=output)

def prepare_data(df):
    """Prepare data for training"""
    # Convert images to array
    X_images = np.array([x for x in df['image']])
    
    # Convert angles to array
    X_angles = np.array([x for x in df['angles']])
    
    # Convert labels to one-hot
    class_to_idx = {cls: i for i, cls in enumerate(df['class'].unique())}
    y = tf.keras.utils.to_categorical(df['class'].map(class_to_idx))
    
    return X_images, X_angles, y, class_to_idx

def train_model():
    # Load processed data
    df = pd.read_pickle('yoga_dataset_processed.pkl')
    
    # Prepare data
    X_img, X_ang, y, class_to_idx = prepare_data(df)
    
    # Split data - USE THE SAME VARIABLE NAMES RETURNED FROM prepare_data()
    (X_img_train, X_img_val, 
     X_ang_train, X_ang_val,
     y_train, y_val) = train_test_split(X_img, X_ang, y, test_size=0.2)
    
    # Create model
    model = create_hybrid_model(num_classes=len(class_to_idx))
    
    # Compile
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Define callbacks
    training_callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=3)
    ]
    
    # Train
    history = model.fit(
        x={'image_input': X_img_train, 'angle_input': X_ang_train},
        y=y_train,
        validation_data=({'image_input': X_img_val, 'angle_input': X_ang_val}, y_val),
        epochs=10,
        batch_size=32,
        callbacks=training_callbacks
    )
    
    # Save class mapping
    idx_to_class = {v:k for k,v in class_to_idx.items()}  # Reverse mapping
    with open('class_mapping.pkl', 'wb') as f:
        pickle.dump(idx_to_class, f)  # New: {0:'downdog', 1:'warrior'}
    
    return model, history

if __name__ == "__main__":
    model, history = train_model()

Epoch 1/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2725 - loss: 23.8504

26/26 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.2735 - loss: 23.7269 - val_accuracy: 0.5789 - val_loss: 3.3711 - learning_rate: 0.0010
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4674 - loss: 8.3821

26/26 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4687 - loss: 8.3107 - val_accuracy: 0.7656 - val_loss: 1.7193 - learning_rate: 0.0010
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6522 - loss: 2.3165

26/26 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6523 - loss: 2.3108 - val_accuracy: 0.7703 - val_loss: 0.9278 - learning_rate: 0.0010
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 941ms/step - accuracy: 0.6678 - loss: 1.5080

26/26 ━━━━━━━━━━━━━━━━━━━━ 26s 991ms/step - accuracy: 0.6676 - loss: 1.5050 - val_accuracy: 0.8134 - val_loss: 0.8242 - learning_rate: 0.0010
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7370 - loss: 1.0694

26/26 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.7361 - loss: 1.0754 - val_accuracy: 0.8230 - val_loss: 0.7302 - learning_rate: 0.0010
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 957ms/step - accuracy: 0.6987 - loss: 1.0751

26/26 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.6990 - loss: 1.0732 - val_accuracy: 0.7847 - val_loss: 0.6441 - learning_rate: 0.0010
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7130 - loss: 0.9713

26/26 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.7132 - loss: 0.9701 - val_accuracy: 0.8182 - val_loss: 0.6299 - learning_rate: 0.0010
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7143 - loss: 0.9178

26/26 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.7146 - loss: 0.9174 - val_accuracy: 0.8373 - val_loss: 0.5666 - learning_rate: 0.0010
Epoch 9/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.7372 - loss: 0.8330 - val_accuracy: 0.8469 - val_loss: 0.6128 - learning_rate: 0.0010
Epoch 10/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7375 - loss: 0.8076

26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7376 - loss: 0.8071 - val_accuracy: 0.8421 - val_loss: 0.5321 - learning_rate: 0.0010


In [3]:

!pip install pyttsx3
!pip install espeak  


ERROR: Could not find a version that satisfies the requirement espeak (from versions: none)
ERROR: No matching distribution found for espeak


In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import pickle
import pyttsx3
import time

# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Voice setup
engine = pyttsx3.init()
engine.setProperty('rate', 150)
last_feedback_time = 0
stable_pose_count = 0
current_pose = None
prev_angles = None

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1, 1)))

def get_angles(landmarks, frame_shape):
    points = {}
    indices = [11,12,13,14,15,16,23,24,25,26,27,28]
    for idx in indices:
        landmark = landmarks[idx]
        points[idx] = (landmark.x * frame_shape[1], landmark.y * frame_shape[0])
    
    angles = [
        calculate_angle(points[11], points[13], points[15]),  # Left elbow
        calculate_angle(points[12], points[14], points[16]),  # Right elbow
        calculate_angle(points[13], points[11], points[23]),  # Left shoulder
        calculate_angle(points[14], points[12], points[24]),  # Right shoulder
        calculate_angle(points[11], points[23], points[25]),  # Left hip
        calculate_angle(points[12], points[24], points[26]),  # Right hip
        calculate_angle(points[23], points[25], points[27]),  # Left knee
        calculate_angle(points[24], points[26], points[28]),  # Right knee
        calculate_angle(points[11], points[23], points[27])   # Spine
    ]
    return points, np.array(angles)

def is_stable(current_angles, prev_angles, threshold=5):
    """Check if pose is stable (angles changed less than threshold)"""
    if prev_angles is None:
        return False
    return np.mean(np.abs(current_angles - prev_angles)) < threshold

def get_pose_feedback(pose_name, angles, confidence):
    global stable_pose_count, current_pose, last_feedback_time, prev_angles
    
    # Stability check
    if not is_stable(angles, prev_angles):
        stable_pose_count = 0
        prev_angles = angles
        return []
    
    stable_pose_count += 1
    prev_angles = angles
    
    # Only give feedback after 3 stable frames and 3 seconds since last feedback
    if stable_pose_count < 3 or (time.time() - last_feedback_time) < 3:
        return []
    
    feedback = []
    if confidence >= 0.99:
        msg = "Excellent form! Maintain this pose."
        feedback.append(msg)
        engine.say(msg)
    else:
        if pose_name == "warrior2":
            if angles[6] < 80:
                msg = "Bend your front knee deeper until thigh is parallel to floor"
                feedback.append(msg)
                engine.say(msg)
            elif angles[6] > 100:
                msg = "Reduce bend in your front knee slightly"
                feedback.append(msg)
                engine.say(msg)
            if angles[7] < 170:
                msg = "Straighten your back leg and press the heel down"
                feedback.append(msg)
                engine.say(msg)
        
        elif pose_name == "tree":
            if angles[7] < 170:
                msg = "Straighten your standing leg more"
                feedback.append(msg)
                engine.say(msg)
    
    if feedback:
        engine.runAndWait()
        last_feedback_time = time.time()
        stable_pose_count = 0  # Reset after feedback
    
    return feedback

def predict_pose(model, img_input, angle_input, idx_to_class):
    pred = model.predict({
        'image_input': np.expand_dims(img_input, axis=0),
        'angle_input': np.expand_dims(angle_input, axis=0)
    }, verbose=0)
    return idx_to_class[np.argmax(pred)], np.max(pred)

def main():
    global current_pose
    
    # Load model
    model = tf.keras.models.load_model('best_model.h5')
    with open('class_mapping.pkl', 'rb') as f:
        idx_to_class = pickle.load(f)
        idx_to_class = {int(k):v for k,v in idx_to_class.items()}

    cap = cv2.VideoCapture(0)
    cv2.namedWindow('Yoga Pose Coach', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Yoga Pose Coach', 1280, 720)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        output = cv2.resize(frame, (1280, 720))
        
        if results.pose_landmarks:
            points, angles = get_angles(results.pose_landmarks.landmark, frame.shape)
            img_input = cv2.resize(image, (224,224)) / 255.0
            pose_name, confidence = predict_pose(model, img_input, angles, idx_to_class)
            
            # Get feedback (with stability checks)
            feedback = get_pose_feedback(pose_name, angles, confidence)
            
            # Display
            y_pos = 50
            cv2.putText(output, f"{pose_name} ({confidence:.2f})", (30,y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 2)
            y_pos += 60
            
            for name, angle in zip(['L-Elbow','R-Elbow','L-Shoulder','R-Shoulder',
                                  'L-Hip','R-Hip','L-Knee','R-Knee','Spine'], angles):
                cv2.putText(output, f"{name}: {angle:.1f}°", (30,y_pos), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
                y_pos += 40
            
            for tip in feedback:
                cv2.putText(output, tip, (30,y_pos), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                y_pos += 50
            
            # Draw scaled landmarks
            for landmark in results.pose_landmarks.landmark:
                landmark.x *= 1280/frame.shape[1]
                landmark.y *= 720/frame.shape[0]
            
            mp_drawing.draw_landmarks(
                output, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=4),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=3)
            )
        
        cv2.imshow('Yoga Pose Coach', output)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    engine.stop()

if __name__ == "__main__":
    main()

NameError: name 'prev_angles' is not defined